## 1. Download training dataset and Flux model files

In [ ]:
import os

flux_lora_training = "./flux_lora_training"
os.makedirs(flux_lora_training, exist_ok=True)

%cd flux_lora_training/

train_image_dir = "./images"
docker_file_dir = "./dockerfile"
os.makedirs(train_image_dir, exist_ok=True)
os.makedirs(docker_file_dir, exist_ok=True)

In [16]:
import json

metadata_file = os.path.join(train_image_dir, 'metadata.jsonl')
with open(metadata_file, 'r', encoding='utf-8') as f:
    for line in f:
        data = json.loads(line)
        filename = data['file_name'].split('.')[0]
        text = data['text']
        output_file = os.path.join(train_image_dir, f'{filename}.txt')
        with open(output_file, 'w', encoding='utf-8') as txt_file:
            txt_file.write(text)
os.remove(metadata_file)

In [ ]:
!pip install --upgrade huggingface_hub

In [ ]:
from huggingface_hub import login, hf_hub_download

# You need to replace below with your own. 
access_token = "******"
login(access_token)

In [ ]:
model_dir = "./models"
os.makedirs(model_dir, exist_ok=True)

model_vae_repo_id = "black-forest-labs/FLUX.1-dev"
model_vae_files = ["flux1-dev.safetensors", "ae.safetensors"]
text_encoders_repo_id = "comfyanonymous/flux_text_encoders"
text_encoders_files = ["clip_l.safetensors", "t5xxl_fp16.safetensors"]

for file in model_vae_files:
    hf_hub_download(model_vae_repo_id, local_dir=model_dir, filename=file)
for file in text_encoders_files:
    hf_hub_download(text_encoders_repo_id, local_dir=model_dir, filename=file)

## 2. Prepare training config files and Dockerfile(docker image for training job)

***Refer "dataset-example.toml" to configure your own .toml file***

In [35]:
%%writefile ./images/dataset.toml
[general]
enable_bucket = true

# DreamBooth caption based datasets
[[datasets]]
resolution = 512
batch_size = 2
caption_extension = '.txt'
keep_tokens = 0

  [[datasets.subsets]]
  image_dir = '/opt/ml/input/data/images/'
  num_repeats = 3

Overwriting ./images/dataset.toml


#### other alternative metadata type

In [36]:
#%%writefile ./images/dataset.toml
#[general]
#enable_bucket = true
#caption_extension = '.txt'
#keep_tokens = 0
#
#[[datasets]]
#resolution = 1024
## min_bucket_reso = 640
## max_bucket_reso = 1536
#bucket_reso_steps = 32
#batch_size = 2
#
#[[datasets.subsets]]
#image_dir = '/opt/ml/input/data/images'

#[general]
#enable_bucket = true
#caption_extension = '.txt'
#keep_tokens = 0
#[[datasets]]
#resolution = [768, 768]
#batch_size = 2
#
#  [[datasets.subsets]]
#  image_dir = '/opt/ml/input/data/images/'
#  metadata_file = '/opt/ml/input/data/images/metadata.jsonl'

In [37]:
#%%writefile ./images/sample_prompt.toml
#[prompt]
#sample_steps = 20
#width = 1024
#height = 1024
#
#[[prompt.subset]]
#prompt = "wta, 1girl, looking at viewer, blue hair, short twintails, hair ornament, blue eyes, blush, smile, open mouth, shirt, skirt, kneehighs, brown footwear, standing, solo"
#seed = 1000
#[[prompt.subset]]
#prompt = "wta, 1girl, looking at viewer, blue hair, short twintails, hair ornament, blue eyes, blush, smile, open mouth, shirt, skirt, kneehighs, brown footwear, standing, solo"
#seed = 2000

In [38]:
%%writefile ./dockerfile/Dockerfile
FROM 763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-training:2.3.0-gpu-py311-cu121-ubuntu20.04-sagemaker

ENV PATH="/opt/ml/code:${PATH}"
ENV SAGEMAKER_SUBMIT_DIRECTORY /opt/ml/code
ENV DEBIAN_FRONTEND noninteractive

RUN git clone -b sd3 https://github.com/kohya-ss/sd-scripts /opt/ml/code

WORKDIR /opt/ml/code

RUN mv flux_train_network.py flux_train_network && \
    sed -i 's/-e \./\./g' requirements.txt && \
    pip3 install torch==2.4.0 torchvision==0.19.0 --index-url https://download.pytorch.org/whl/cu124 && \
    pip install -r requirements.txt && \
    pip install wandb && \
    pip uninstall transformer-engine -y # Solve error of "transformer_engine_extensions.cpython-311-x86_64-linux-gnu.so: undefined symbol"

# RUN mkdir -p images/

# COPY ./images/* ./images/

WORKDIR /

ENV SAGEMAKER_PROGRAM accelerate.commands.launch --mixed_precision bf16 --num_cpu_threads_per_process 1 flux_train_network

Writing ./dockerfile/Dockerfile


## 3. Build docker image and push to ECR

In [ ]:
## You should change below region code to the region you used, here sample is use us-west-2
!aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-west-2.amazonaws.com

In [ ]:
%%sh
algorithm_name=flux-lora-taining-job

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

#load public ECR image
#aws ecr-public get-login-password --region us-east-1 | docker login --username AWS --password-stdin public.ecr.aws

# Log into Docker
pwd=$(aws ecr get-login-password --region ${region})
docker login --username AWS -p ${pwd} ${account}.dkr.ecr.${region}.amazonaws.com

docker build -t ${algorithm_name} ./dockerfile
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

## 5. Train models with SageMaker training job

In [39]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()
account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name
images_s3uri = 's3://{0}/flux-lora-train/dataset/'.format(bucket)
models_s3uri = 's3://{0}/flux-lora-train/models/'.format(bucket)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [40]:
# Copy training dataset to S3 bucket

!aws s3 cp images $images_s3uri --recursive

upload: images/.ipynb_checkpoints/metadata-checkpoint.jsonl to s3://sagemaker-us-west-2-687912291502/flux-lora-train/dataset/.ipynb_checkpoints/metadata-checkpoint.jsonl
upload: images/2.txt to s3://sagemaker-us-west-2-687912291502/flux-lora-train/dataset/2.txt
upload: images/4.txt to s3://sagemaker-us-west-2-687912291502/flux-lora-train/dataset/4.txt
upload: images/5.jpg to s3://sagemaker-us-west-2-687912291502/flux-lora-train/dataset/5.jpg
upload: images/.ipynb_checkpoints/5-checkpoint.jpg to s3://sagemaker-us-west-2-687912291502/flux-lora-train/dataset/.ipynb_checkpoints/5-checkpoint.jpg
upload: images/6.txt to s3://sagemaker-us-west-2-687912291502/flux-lora-train/dataset/6.txt
upload: images/5.txt to s3://sagemaker-us-west-2-687912291502/flux-lora-train/dataset/5.txt
upload: images/.ipynb_checkpoints/dataset-checkpoint.toml to s3://sagemaker-us-west-2-687912291502/flux-lora-train/dataset/.ipynb_checkpoints/dataset-checkpoint.toml
upload: images/3.txt to s3://sagemaker-us-west-2-687

In [41]:
# Copy flux model files to S3 bucket

!aws s3 cp models $models_s3uri --recursive


The user-provided path models does not exist.


***You need to provide your own "wandb_api_key" for below scripts***

In [57]:
import json
def json_encode_hyperparameters(hyperparameters):
    for (k, v) in hyperparameters.items():
        print(k, v)
    return {k: json.dumps(v) for (k, v) in hyperparameters.items()}

docker_image_uri = '{0}.dkr.ecr.{1}.amazonaws.com/flux-lora-taining-job'.format(account_id, region_name)
instance_type = 'ml.g5.2xlarge'

lora_name = "flux_lora_wta"
output_dir="/opt/ml/model/"
wandb_api_key = "*******" # Provide your wandb key

environment = {'LD_LIBRARY_PATH': "${LD_LIBRARY_PATH}:/opt/conda/lib/python3.11/site-packages/nvidia/nvjitlink/lib/"}

hyperparameters = {
                    'pretrained_model_name_or_path': '/opt/ml/input/data/models/flux1-dev.safetensors',
                    'clip_l': '/opt/ml/input/data/models/clip_l.safetensors',
                    't5xxl': '/opt/ml/input/data/models/t5xxl_fp16.safetensors',
                    'ae': '/opt/ml/input/data/models/ae.safetensors',
                    'save_model_as': 'safetensors',
                    'sdpa': '',
                    'persistent_data_loader_workers': '',
                    'max_data_loader_n_workers': 2,
                    'gradient_checkpointing': '',
                    'mixed_precision': 'bf16',
                    'save_precision': 'bf16',
                    'full_bf16': '',
                    'network_module': 'networks.lora_flux',
                    'network_dim': 64,
                    'network_alpha': 32,
                    'lr_scheduler': 'cosine_with_restarts',
                    'lr_scheduler_num_cycles': 1,
                    'optimizer_type': 'prodigy',
                    'optimizer_args': 'safeguard_warmup=True',
                    'learning_rate': 1e-4,
                    'cache_latents_to_disk': '',
                    'cache_text_encoder_outputs': '',
                    'cache_text_encoder_outputs_to_disk': '',
                    'fp8_base': '',
                    'highvram': '',
                    'max_train_steps': 540,
                    'save_every_n_steps': 120,
                    'dataset_config': '/opt/ml/input/data/images/dataset.toml',
                    'output_dir': output_dir,
                    'output_name': lora_name,
                    'timestep_sampling': 'shift',
                    'discrete_flow_shift': 3.1582,
                    'model_prediction_type': 'raw',
                    'guidance_scale': 1,
                    't5xxl_max_token_length': 512,
                    #'sample_every_n_steps': 120,
                    #'sample_prompts': '/opt/ml/input/data/images/sample_prompt.toml',
                    'sample_sampler': 'euler_a',
                    'logging_dir': '/opt/ml/code/logs',
                    'log_with': 'all',
                    'log_tracker_name': lora_name,
                    'log_config':'',
                    'wandb_api_key': wandb_api_key
}

hyperparameters = json_encode_hyperparameters(hyperparameters)

pretrained_model_name_or_path /opt/ml/input/data/models/flux1-dev.safetensors
clip_l /opt/ml/input/data/models/clip_l.safetensors
t5xxl /opt/ml/input/data/models/t5xxl_fp16.safetensors
ae /opt/ml/input/data/models/ae.safetensors
save_model_as safetensors
sdpa 
persistent_data_loader_workers 
max_data_loader_n_workers 2
gradient_checkpointing 
mixed_precision bf16
save_precision bf16
full_bf16 
network_module networks.lora_flux
network_dim 64
network_alpha 32
lr_scheduler cosine_with_restarts
lr_scheduler_num_cycles 1
optimizer_type prodigy
optimizer_args safeguard_warmup=True
learning_rate 0.0001
cache_latents_to_disk 
cache_text_encoder_outputs 
cache_text_encoder_outputs_to_disk 
fp8_base 
highvram 
max_train_steps 540
save_every_n_steps 120
dataset_config /opt/ml/input/data/images/dataset.toml
output_dir /opt/ml/model/
output_name flux_lora_wta
timestep_sampling shift
discrete_flow_shift 3.1582
model_prediction_type raw
guidance_scale 1
t5xxl_max_token_length 512
sample_sampler eule

In [58]:
from sagemaker.estimator import Estimator

inputs = {
    'images': images_s3uri,
    'models': models_s3uri
}

estimator = Estimator(
    role = role,
    instance_count=1,
    instance_type = instance_type,
    image_uri = docker_image_uri,
    hyperparameters = hyperparameters,
    environment=environment,
    disable_output_compression = True,
    sagemaker_session=sagemaker_session,
    enable_remote_debug=True,
    
)
estimator.fit(inputs=inputs,wait=False)

INFO:sagemaker:Creating training-job with name: flux-lora-taining-job-2024-09-01-14-34-16-619


In [59]:
### remote debug连接training job ssm容器实例
job_name="flux-lora-taining-job-2024-09-01-13-36-04-512"
training_job_info = sagemaker_session.describe_training_job(job_name)
print(training_job_info)
#!aws ssm start-session --target sagemaker-training-job:${job_name}_algo-1

{'TrainingJobName': 'flux-lora-taining-job-2024-09-01-13-36-04-512', 'TrainingJobArn': 'arn:aws:sagemaker:us-west-2:687912291502:training-job/flux-lora-taining-job-2024-09-01-13-36-04-512', 'ModelArtifacts': {'S3ModelArtifacts': 's3://sagemaker-us-west-2-687912291502/flux-lora-taining-job-2024-09-01-13-36-04-512/output/model'}, 'TrainingJobStatus': 'Completed', 'SecondaryStatus': 'Completed', 'HyperParameters': {'ae': '"/opt/ml/input/data/models/ae.safetensors"', 'cache_latents_to_disk': '""', 'cache_text_encoder_outputs': '""', 'cache_text_encoder_outputs_to_disk': '""', 'clip_l': '"/opt/ml/input/data/models/clip_l.safetensors"', 'dataset_config': '"/opt/ml/input/data/images/dataset.toml"', 'discrete_flow_shift': '3.1582', 'fp8_base': '""', 'full_bf16': '""', 'gradient_checkpointing': '""', 'guidance_scale': '1', 'highvram': '""', 'learning_rate': '1.0', 'log_config': '""', 'log_tracker_name': '"flux_lora_wta"', 'log_with': '"all"', 'logging_dir': '"/opt/ml/code/logs"', 'lr_scheduler'

In [55]:
model_data = estimator.model_data
model_s3_path = model_data['S3DataSource']['S3Uri']
print("Model artifact saved at:", "\n"+model_s3_path+"\n")
!aws s3 ls {model_s3_path}

Model artifact saved at: 
s3://sagemaker-us-west-2-687912291502/flux-lora-taining-job-2024-09-01-13-36-04-512/output/model/

2024-09-01 14:32:08  634008304 flux_lora_wta-step00000120.safetensors
2024-09-01 14:32:10  634008304 flux_lora_wta-step00000240.safetensors
2024-09-01 14:32:06  634008304 flux_lora_wta-step00000360.safetensors
2024-09-01 14:32:12  634008304 flux_lora_wta-step00000480.safetensors
2024-09-01 14:32:17  634008304 flux_lora_wta.safetensors


In [56]:
# You can change the applied lora weight by changing lora weight name

lora_s3_path = model_s3_path + 'flux_lora_wta.safetensors'
print ("Lora weight is saved at:", "\n"+lora_s3_path)

Lora weight is saved at: 
s3://sagemaker-us-west-2-687912291502/flux-lora-taining-job-2024-09-01-13-36-04-512/output/model/flux_lora_wta.safetensors
